In [1]:
import jax
import sys
sys.path.append("..")
from jaxrl_m.typing import *
from jaxrl_m.networks import MLP, get_latent, default_init, ensemblize
import equinox as eqx
import equinox.nn as nn
import functools

In [2]:
import warnings
warnings.filterwarnings('ignore')
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import jax
import jax.numpy as jnp
import equinox as eqx
import equinox.nn as nn
import optax

import ott
from ott.solvers.linear import sinkhorn
from ott.geometry import pointcloud

import matplotlib.pyplot as plt
import pprint


In [3]:
from src.special_networks import MultilinearVF_EQX
rngs = jax.random.PRNGKey(42)
key1, key2 = jax.random.split(rngs, 2)
import functools
hidden_dims=[256, 256]
network_cls = functools.partial(nn.MLP, in_size=29, out_size=hidden_dims[-1],
                                        width_size=hidden_dims[0], depth=len(hidden_dims),
                                        final_activation=jax.nn.relu)

phi_net = network_cls(key=key1)

In [4]:
loaded_phi_net = eqx.tree_deserialise_leaves("/home/m_bobrin/GOTIL/icvf_phi_300k_umaze.eqx", phi_net)

In [6]:
@eqx.filter_vmap
def ensemblize(keys):
    return MultilinearVF_EQX(key=keys, state_dim=29, hidden_dims=[256, 256], pretrained_phi=loaded_phi_net)

ensemble_net = ensemblize(jax.random.split(key1, 2))